In [7]:
from rk_methods import *
from imex_methods import *
from measure import *

def create_mesasurement(f, g, A, A_, b, b_, c, Tl, Tr, f_exact, errors, G = None, N_min = 2000, N_max = 10000):
    data = None
    header = ['Name', 'Exec time (s)', 'Step Size', 'Error']
    for error in errors:
        print(f'Calculating for error: {error}')
        if G is None:
            starting_method = lambda y0, Tl, Tr, N: IMEX(f, ImplicitSolver(g), y0, A, A_, b, b_, c, Tl, Tr, N) 
        else:
            starting_method = lambda y0, Tl, Tr, N: IMEX(f, LinearImplicitSolver(G), y0, A, A_, b, b_, c, Tl, Tr, N)

        test_solve = measure_method(starting_method, 'Test', f_exact(Tl), Tl, Tr, f_exact, error, N_min, N_min, N_max)
        N_ = test_solve.N

        f_ = lambda t, x: f(t, x) + g(t, x)
        c_ = [0] + c
        methods = [
            (lambda y0, Tl, Tr, N: ERK(f_, y0, A_, b_, c_, Tl, Tr, N), 'ERK'),
            (lambda y0, Tl, Tr, N: IRK(f_, y0, A, b, c, Tl, Tr, N), 'IRK'),
            (lambda y0, Tl, Tr, N: IMEX(f, ImplicitSolver(g), y0, A, A_, b, b_, c, Tl, Tr, N), 'IMEX'),
        ]
        if G is not None:
            methods += [
                (lambda y0, Tl, Tr, N: IMEX(f, LinearImplicitSolver(G), y0, A, A_, b, b_, c, Tl, Tr, N), 'IMEX Lin'),
                (lambda y0, Tl, Tr, N: IMEX(f, LinearImplicitSolverLU(G), y0, A, A_, b, b_, c, Tl, Tr, N), 'IMEX LU')
            ]
        
        results = []
        for method, name in methods:
            print(f'Measuring method: {name}')
            results.append(measure_method(method, name, f_exact(Tl), Tl, Tr, f_exact, error, N_, N_min, N_max))

        data_ = get_table_data(results)

        if data is None:
            data = data_
        else:
            for i in range(len(data)):
                data[i] = data[i] + data_[i][1:]
                header += ['Exec time (s)', 'Step Size', 'Error']

    table = tabulate.tabulate(data, headers=header, tablefmt='html')
    return table

# IMEX Euler

$$
    y_{n+1} = y_n + h ( f_1(t_n, y_n) + f_2(t_{n + 1}, y_{n+1}))
$$

In [8]:
from rk_methods import *
from imex_methods import *

IMEX_EULER = lambda f, g, u0, Tl, Tr, N: IMEX(f, g, u0, [[1]], [[0, 0],[1, 0]], [1], [1, 0], [1], Tl, Tr, N)
IMEX_EULER2 = lambda f, g, u0, Tl, Tr, N: IMEX(f, g, u0, [[1]], [[0, 0],[1, 0]], [1], [0, 1], [1], Tl, Tr, N)

In [9]:
def g(t, u):
    return -100 * u

def f(t, u):
    return 10 * np.cos(t)

def f_exact(t):
    return 1/10001 * (9001 * np.exp(-100 * t) + 10 * np.sin(t) + 1000 * np.cos(t))

In [10]:
A = [[1]]
b = [1]
c = [1]
A_ = [[0, 0], [1, 0]]
b_ = [1, 0]

table = create_mesasurement(f, g, A, A_, b, b_, c, 0, 20, f_exact, [0.1, 0.05, 0.025], [[-100]], 2000, 15000)

from IPython.display import HTML, display
display(HTML(table))

Calculating for error: 0.1
Measuring method: ERK
Measuring method: IRK
Measuring method: IMEX
Measuring method: IMEX Lin
Measuring method: IMEX LU
Calculating for error: 0.05
Measuring method: ERK
Measuring method: IRK
Measuring method: IMEX
Measuring method: IMEX Lin
Measuring method: IMEX LU
Calculating for error: 0.025
Measuring method: ERK
Measuring method: IRK
Measuring method: IMEX
Measuring method: IMEX Lin
Measuring method: IMEX LU


Name,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error
ERK,0.098,0.00475,0.09999,0.21111,0.00269,0.05,0.83013,0.00142,0.025
IRK,0.59285,0.00835,0.09998,1.0471,0.00344,0.05,2.88796,0.00161,0.025
IMEX,0.2907,0.00835,0.09999,0.54007,0.00344,0.05,1.11985,0.00161,0.025
IMEX Lin,0.21111,0.00835,0.09999,0.38751,0.00344,0.05,0.79686,0.00161,0.025
IMEX LU,0.26682,0.00835,0.09999,0.69263,0.00344,0.05,1.15255,0.00161,0.025


In [11]:
from order_estimate import *

IMEX_E_order_result = order_from_fine_grid(lambda y0, Tl, Tr, N: IMEX_EULER(f, LinearImplicitSolver([[-100]]), y0, Tl, Tr, N), f_exact(0), 0, 20, [2**11, 2**12, 2**13], 2**16)
print('Estimating order from fine grid:')
print(f'Order result: {IMEX_E_order_result}')

Estimating order from fine grid:
Order result: [0.83634028 0.97674382]


# IMEX trapéz módszer